In [ ]:
# ============================================================
# 📘 Pipeline explanation: From SMILES to commercial availability
#
# This collection of blocks forms a complete workflow that:
# 1. Generates InChIKeys from SMILES using RDKit.
# 2. Queries PubChem with its API (pubchempy) to retrieve CIDs/SIDs for each compound.
# 3. Retrieves vendor information and commercial availability.
#
# ✅ Requirements and recommendations:
# - The input Excel file must contain a column named "Smiles".
# - It is highly recommended to include an "ID" column
#   (unique identifier for each compound).
# - If the database already contains calculated columns
#   ("InChIKey", "PubChem CID/SID", etc.), the script will
#   skip those entries. This allows updating existing datasets
#   without overwriting previous results.
#
# ⚡ Workflow robustness:
# - Each block automatically resumes from existing output files
#   (e.g., *_inchikey.xlsx, *_CID.xlsx, *_vendors.xlsx), including partially completed runs.
# - You can optionally resume from a specific compound ID in blocks that support it
#   (e.g., InChIKey generation: start_from_id = 'LANaPDB13579').
# - Safe to interrupt: results are progressively saved and
#   can be resumed in the next execution.
# - ⚠️ Important: when starting the pipeline from scratch, do NOT use a file
#   that already has suffixes like "_inchikey", "_CID", or "_vendors" as input,
#   to avoid confusion in later blocks.
#
# ------------------------------------------------------------
# Example 1: Minimal input database (only ID and Smiles)
# (first 5 rows)
#     ID      Smiles
#     001     CCO
#     002     C1CCO1
#     003     CCN(CC)C
#     004     COC
#     005     CC(=O)O
#
# ------------------------------------------------------------
# Example 2: Database that can be updated with this workflow
# (some columns already filled, others empty)
#     ID      Smiles        InChIKey                           PubChem CID/SID   Number of vendors in Pubchem   Commercial availability
#     001     CCO           LFQSCWFLJHTTHZ-UHFFFAOYSA-N             702                 12                             Purchasable
#     002     C1CCO1        ZUXXECWHPGJHCN-UHFFFAOYSA-N             1234                13                             Not purchasable
#     003     CCN(CC)C      KXKPIXYAXDZYMO-UHFFFAOYSA-N             325                 20                             Purchasable
#     004     COC           <empty>                                 <empty>            <empty>                         <empty>
#     005     CC(=O)O       <empty>                                 <empty>            <empty>                         <empty>
#
# In this second case, the workflow will:
# - Skip already completed values (e.g., row 001).
# - Resume missing calculations for the empty fields.
# ============================================================


In [ ]:
# ============================================================
# 🔑 InChIKey generation from SMILES
#
# This block reads the input Excel file (must contain a column
# named "Smiles") and generates a standardized identifier
# (InChIKey) for each molecule using RDKit.
#
# ✅ Behavior:
# - If 'InChIKey' does not exist, it is created and filled.
# - If 'InChIKey' exists, only empty/error cells are recalculated.
# - If SMILES is invalid, assigns "error".
# - Can resume from a given ID to continue work.
# ============================================================

from rdkit import Chem
from rdkit.Chem import inchi
import pandas as pd
import os

# ===================== CONFIGURATION =====================
input_file = 'Type here the name of the input file.xlsx'   
# 📂 Input file (must be the original SMILES dataset)
# ⚠️ Important: Do NOT use a file that already contains a suffix like "_inchikey", "_CID", or "_vendors"
#    from previous pipeline steps, as this may cause incorrect processing or duplicated data.

# 🔁 Optionally, resume from a specific compound ID:
#    - The ID must be enclosed in quotes (e.g., 'LANaPDB13579')
#    - If None, the script will start from the beginning
start_from_id = None   # Example: start_from_id = 'LANaPDB13579'
# =========================================================


# Read Excel file
df = pd.read_excel(input_file)

# Add index column if not present
if 'Index' not in df.columns:
    df['Index'] = df.reset_index().index

# Ensure 'InChIKey' column exists
if 'InChIKey' not in df.columns:
    df['InChIKey'] = pd.Series(dtype="object")

# Filter if resuming from a given ID
if start_from_id is not None:
    df_to_process = df[df['Index'] >= start_from_id]
    print(f"⏩ Resuming InChIKey generation from Index {start_from_id} ...")
else:
    df_to_process = df
    print("▶️ Starting InChIKey generation from the beginning...")

# Process SMILES → InChIKey
for idx, row in df_to_process.iterrows():
    if pd.isna(row['Smiles']) or row['Smiles'] == "":
        continue  # skip empty SMILES

    if pd.isna(row['InChIKey']) or row['InChIKey'] in ["", "error"]:
        try:
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is not None:
                df.at[idx, 'InChIKey'] = inchi.MolToInchiKey(mol)
            else:
                df.at[idx, 'InChIKey'] = "error"
        except:
            df.at[idx, 'InChIKey'] = "error"

# Generate output filename
file_root, file_ext = os.path.splitext(input_file)
output_file = f"{file_root}_inchikey{file_ext}"

# Save updated file
df.to_excel(output_file, index=False)

print(f"\n✅ File saved as: {output_file}")


In [ ]:
# ============================================================
# 🔍 PubChem CID Retrieval (batched, resumable, safe interrupt)
#
# Reads the *_inchikey.xlsx file and queries PubChem
# to retrieve the CID of each compound.
#
# ✅ Key features:
# - Real-time progress (progress bar, estimated remaining time, last successful ID).
# - Batch processing → safe to interrupt in Jupyter.
# - Automatic saving after each batch.
# - Clear messages on manual interruption.
# - Auto-resume if *_CID.xlsx already exists.
# ============================================================

import glob
import os
import pandas as pd
import pubchempy as pcp
import time
import math

# ===================== CONFIGURATION =====================
batch_size = 50         # ⚡ Process in small batches → safe to interrupt
max_retries = 3         # 🔄 Retry attempts per compound
bar_length = 30         # 📊 Progress bar length
# =========================================================

# Detect *_inchikey.xlsx file
inchikey_files = glob.glob('*_inchikey.xlsx')
if not inchikey_files:
    raise FileNotFoundError("❌ No *_inchikey.xlsx file found in the current folder.")
inchikey_file = inchikey_files[0]

# Define output file
file_root, file_ext = os.path.splitext(inchikey_file)
output_file = f"{file_root}_CID{file_ext}"

# Load dataframe (resume if previous file exists)
if os.path.exists(output_file):
    df = pd.read_excel(output_file)
    print(f"📂 Resuming from existing CID file: {output_file}")
else:
    df = pd.read_excel(inchikey_file)
    for col in ["PubChem CID/SID", "PubChem others CID/SID", "InChIKey"]:
        if col not in df.columns:
            df[col] = pd.Series(dtype=object)
    print(f"📂 No results file found. Starting from input file: {inchikey_file}")
    # 👉 Ensure the output file exists from the start
    df.to_excel(output_file, index=False)

# Force object dtype (avoids warnings)
df["PubChem CID/SID"] = df["PubChem CID/SID"].astype(object)
df["PubChem others CID/SID"] = df["PubChem others CID/SID"].astype(object)

# ===================== FUNCTIONS =====================
def fetch_cid(inchikey, retries=max_retries):
    """Query PubChem with retries. Returns (CID, others)."""
    if pd.isna(inchikey) or inchikey in ["", "error"]:
        return "InChIKey not found", None
    for attempt in range(1, retries + 1):
        try:
            compounds = pcp.get_compounds(inchikey, "inchikey")
            if not compounds:
                return "CID not found", None
            if len(compounds) == 1:
                return str(compounds[0].cid), None
            else:
                first = str(compounds[0].cid)
                others = ",".join(str(c.cid) for c in compounds[1:])
                return first, others
        except Exception:
            if attempt < retries:
                time.sleep(1)
                continue
            return "CID not found", None
    return "CID not found", None

def print_progress(done, total, start_time, last_id):
    """Print progress bar with estimated remaining time."""
    elapsed = time.time() - start_time
    avg = elapsed / done if done > 0 else 0
    remaining = total - done
    estimated_time = avg * remaining
    h, m, s = int(estimated_time // 3600), int((estimated_time % 3600) // 60), int(estimated_time % 60)

    ratio = done / total
    filled = math.ceil(bar_length * ratio)
    bar = "#" * filled + "-" * (bar_length - filled)
    percent = ratio * 100

    print(
        f"\r[{bar}] {done}/{total} ({percent:.1f}%) "
        f"| Remaining: {remaining} "
        f"| Last successful ID: {last_id if last_id else '—'} "
        f"| Estimated remaining time: {h}h {m}m {s}s",
        end="", flush=True
    )

# ===================== MAIN LOOP =====================
unprocessed = df[df["PubChem CID/SID"].isna() | (df["PubChem CID/SID"] == "")]
if unprocessed.empty:
    print("✅ All compounds are already processed. Skipping step.")
    # 👉 Ensure output file exists
    df.to_excel(output_file, index=False)
else:
    total = len(df)
    already_done = (df["PubChem CID/SID"].notna() & (df["PubChem CID/SID"] != "")).sum()
    remaining = total - already_done

    print(f"🔎 Compounds pending PubChem CID retrieval: {remaining}")

    processed_this_run = 0
    start_time = time.time()
    last_successful_id = None

    try:
        for start in range(0, len(unprocessed), batch_size):
            batch = unprocessed.iloc[start:start+batch_size]
            for idx, row in batch.iterrows():
                cid_val, others_val = fetch_cid(row["InChIKey"])
                df.at[idx, "PubChem CID/SID"] = cid_val
                df.at[idx, "PubChem others CID/SID"] = others_val if others_val else ""

                already_done += 1
                processed_this_run += 1

                if cid_val not in ["CID not found", "InChIKey not found"]:
                    last_successful_id = df.at[idx, "ID"] if "ID" in df.columns else f"Index {idx}"

                print_progress(already_done, total, start_time, last_successful_id)

            # Save after each batch
            df.to_excel(output_file, index=False)

    except KeyboardInterrupt:
        print(f"\n🛑 Interrupted by user. Saving progress up to ID: {last_successful_id if last_successful_id else '—'}...")
        df.to_excel(output_file, index=False)
        print(f"💾 Progress saved in {output_file}")
        raise

    # Final save
    df.to_excel(output_file, index=False)
    print(f"\n💾 Final results saved in {output_file}")
    print(f"✅ Total processed: {already_done}")
    print(f"   ❌ CID not found: {(df['PubChem CID/SID'] == 'CID not found').sum()}")
    print(f"   ❌ InChIKey not found: {(df['PubChem CID/SID'] == 'InChIKey not found').sum()}")


In [ ]:
# ============================================================
# 🛒 Obtain Commercial Availability from PubChem (parallelized, safe interrupt)
#
# This block uses the *_CID.xlsx file as input and queries PubChem
# for vendor information.
#
# ✅ Key features:
# - Safe to interrupt: progress is automatically saved on manual stop.
# - Automatic resume: resumes from *_vendors.xlsx if it exists.
# - Parallelized requests (default: 30 workers, safe range 10–50)
# - Batches to allow fast interruption.
# ============================================================

import pandas as pd
import requests
import glob
import os
import time
import math
from concurrent.futures import ThreadPoolExecutor, as_completed

# ===================== CONFIGURATION =====================
save_every = 5000      # 💾 Save progress every X compounds
max_workers = 30       # ⚡ Parallel requests
max_retries = 3        # 🔄 Number of retries per failed request
retry_delay = 2        # ⏱ Seconds to wait between retries
batch_size = 50        # Number of compounds per batch (for safe interrupt)
request_timeout = 10   # Timeout for PubChem requests
bar_length = 30        # Progress bar length
# =========================================================

# Detect *_CID.xlsx file automatically
cid_files = glob.glob('*_CID.xlsx')
if not cid_files:
    raise FileNotFoundError("No *_CID.xlsx file found in the current folder.")
cid_file = cid_files[0]

# Output filename
file_root, file_ext = os.path.splitext(cid_file)
output_file = f"{file_root}_vendors{file_ext}"

# Load dataframe (resume if vendors file exists)
if os.path.exists(output_file):
    df = pd.read_excel(output_file)
    print(f"📂 Resuming from existing vendors file: {output_file}")
else:
    df = pd.read_excel(cid_file)
    for col in ['Number of vendors in Pubchem', 'Commercial availability']:
        if col not in df.columns:
            df[col] = pd.Series(dtype='object')
    print(f"📂 Starting new vendors calculation from CID file: {cid_file}")

# Force object type to avoid Pandas warnings
df['Number of vendors in Pubchem'] = df['Number of vendors in Pubchem'].astype(object)
df['Commercial availability'] = df['Commercial availability'].astype(object)

# Add index if missing
if 'Index' not in df.columns:
    df['Index'] = df.reset_index().index

# Identify unprocessed compounds
unprocessed_idx = df[(df['PubChem CID/SID'].notna()) &
                     (df['PubChem CID/SID'] != "") &
                     ((df['Number of vendors in Pubchem'].isna()) |
                      (df['Number of vendors in Pubchem'] == ""))].index

if unprocessed_idx.empty:
    print("✅ All compounds have already been processed. Nothing to do.")
else:
    total_compounds = len(df)
    already_done = total_compounds - len(unprocessed_idx)
    processed_this_run = 0
    start_time = time.time()
    last_success_id = "N/A"

    print(f"\n📊 Resume summary:")
    print(f"   Already processed: {already_done}")
    print(f"   Remaining: {len(unprocessed_idx)}")
    print(f"   Using {max_workers} parallel requests (safe range: 10–50).")
    print(f"   Each request will retry up to {max_retries} times if it fails.\n")

    # ===================== Process function =====================
    def process_compound(idx):
        """Fetch vendor info for a given compound CID."""
        cid = df.at[idx, 'PubChem CID/SID']
        result = {"idx": idx, "vendors_count": 0, "availability": "Not purchasable", "success": False}

        if pd.isna(cid) or cid in ["", "CID not found"]:
            result["vendors_count"] = "CID missing"
            result["availability"] = "Not purchasable"
            return result

        for attempt in range(max_retries):
            try:
                response = requests.get(
                    f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/categories/compound/{cid}/JSON/?response_type=display',
                    timeout=request_timeout
                )
                lines = response.text.splitlines()
                source_lines = [line.replace('"SourceName":', "").strip() for line in lines if "SourceName" in line]
                vendors = set(source_lines)
                result["vendors_count"] = len(vendors)
                result["availability"] = "Purchasable" if len(vendors) > 0 else "Not purchasable"
                result["success"] = True
                return result
            except Exception:
                if attempt < max_retries - 1:
                    time.sleep(retry_delay)
                else:
                    result["vendors_count"] = "Error"
                    result["availability"] = "Error"
        return result

    # ===================== Main Loop =====================
    try:
        for batch_start in range(0, len(unprocessed_idx), batch_size):
            batch_idx = unprocessed_idx[batch_start:batch_start+batch_size]
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = {executor.submit(process_compound, idx): idx for idx in batch_idx}

                for future in as_completed(futures):
                    res = future.result()
                    idx = res["idx"]
                    df.at[idx, 'Number of vendors in Pubchem'] = res["vendors_count"]
                    df.at[idx, 'Commercial availability'] = res["availability"]

                    processed_this_run += 1
                    already_done += 1
                    remaining = len(unprocessed_idx) - processed_this_run

                    if res["success"]:
                        last_success_id = df.at[idx, 'ID'] if 'ID' in df.columns else "N/A"

                    # Time left calculation
                    elapsed_time = time.time() - start_time
                    avg_time_per = elapsed_time / processed_this_run if processed_this_run > 0 else 0
                    remaining_seconds = avg_time_per * remaining
                    eta_h, eta_m = int(remaining_seconds // 3600), int((remaining_seconds % 3600) // 60)
                    eta_s = int(remaining_seconds % 60)

                    # Progress bar
                    progress_ratio = already_done / total_compounds
                    filled_len = math.ceil(bar_length * progress_ratio)
                    bar = '#' * filled_len + '-' * (bar_length - filled_len)

                    next_save_in = save_every - (processed_this_run % save_every)
                    if next_save_in == save_every:
                        next_save_in = 0

                    print(
                        f"\r[{bar}] {already_done}/{total_compounds} ({progress_ratio*100:.1f}%) "
                        f"| Already processed: {already_done} | Remaining: {remaining} "
                        f"| Last successful processed ID: {last_success_id} "
                        f"| Next save in: {next_save_in} "
                        f"| Time left: {eta_h}h {eta_m}m {eta_s}s",
                        end='', flush=True
                    )

                    # Save every batch or save_every compounds
                    if processed_this_run % save_every == 0:
                        print(f"\n🛑 Saving progress...")
                        df.to_excel(output_file, index=False)
                        print(f"✅ Progress saved in {output_file}")

    except KeyboardInterrupt:
        print(f"\n🛑 Manual interruption detected! Saving progress up to ID: {last_success_id}...")
        df.to_excel(output_file, index=False)
        print(f"✅ Progress successfully saved in {output_file}")
        raise

    # Final save
    if 'Index' in df.columns:
        df.drop(columns=['Index'], inplace=True)
    df.to_excel(output_file, index=False)
    print(f"\n✅ Final file saved as: {output_file}")


In [ ]:
# ============================================================
# 📊 Summary of Commercial Availability and PubChem Data
#
# This block reads the latest *_vendors.xlsx file generated previously
# and provides a summary of the compounds in terms of:
# 1. Commercial availability status (Purchasable / Not purchasable / CID missing).
# 2. PubChem CID/SID presence (found or not found).
# 3. Compounds with 0 vendors but a valid CID/SID.
#
# ✅ Key points:
# - Automatically detects the latest *_vendors.xlsx file in the folder.
# - Calculates counts and percentages for each category.
# - Useful for a quick overview of the dataset after all previous
#   processing steps (InChIKey, PubChem CID, vendor retrieval) are complete.
# ============================================================

import pandas as pd
import glob

# Automatically detect the latest vendors file
vendors_files = glob.glob('*_vendors.xlsx')
if not vendors_files:
    raise FileNotFoundError("No *_vendors.xlsx file found in the current folder.")
input_file = vendors_files[0]
print(f"Using input file: {input_file}\n")

# Load the file
df = pd.read_excel(input_file)
total_compounds = len(df)

# ------------------ Commercial availability ------------------
if 'Commercial availability' in df.columns:
    ca_counts = df['Commercial availability'].replace({'CID missing':'Not purchasable'}).value_counts()
    print(f"{'Commercial availability':<35}{'Number of compounds':<20}{'Percentage':<10}")
    for status, count in ca_counts.items():
        percent = (count / total_compounds) * 100
        print(f"{status:<35}{count:<20}{percent:.2f}%")

print()  # blank line for separation

# ------------------ PubChem CID/SID presence ------------------
if 'PubChem CID/SID' in df.columns:
    cid_found_count = df['PubChem CID/SID'].apply(lambda x: 'CID not found' not in str(x)).sum()
    cid_not_found_count = total_compounds - cid_found_count
    print(f"{'PubChem CID/SID':<35}{'Number of compounds':<20}{'Percentage':<10}")
    print(f"{'CID/SID found':<35}{cid_found_count:<20}{(cid_found_count/total_compounds)*100:.2f}%")
    print(f"{'CID/SID not found':<35}{cid_not_found_count:<20}{(cid_not_found_count/total_compounds)*100:.2f}%")

print()  # blank line for separation

# ------------------ Compounds with 0 vendors but valid CID/SID ------------------
if 'Number of vendors in Pubchem' in df.columns and 'PubChem CID/SID' in df.columns:
    zero_vendor_valid_cid = df[
        (df['Number of vendors in Pubchem'] == 0) &
        (~df['PubChem CID/SID'].isin(['CID not found', 'CID missing']))
    ]
    count_zero_vendor_valid_cid = len(zero_vendor_valid_cid)
    percent_zero_vendor_valid_cid = (count_zero_vendor_valid_cid / total_compounds) * 100
    print(f"{'Zero vendors but valid CID/SID':<35}{'Number of compounds':<20}{'Percentage':<10}")
    print(f"{'':<35}{count_zero_vendor_valid_cid:<20}{percent_zero_vendor_valid_cid:.2f}%")
